In [ ]:
import tensorflow as tf
assert tf.__version__.startswith('2')

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.callbacks import TensorBoard

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
tf.__version__

'2.3.0'

In [ ]:
zip_file = tf.keras.utils.get_file(origin='//content/drive/My Drive/Dataset/Dataset', fname="/content/drive/My Drive/Dataset/Dataset")

In [ ]:
base_dir = os.path.join(os.path.dirname(zip_file), 'Dataset')

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation')

Found 1653 images belonging to 10 classes.
Found 409 images belonging to 10 classes.


In [ ]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 10))

In [ ]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


In [ ]:
!cat labels.txt

0
1
2
3
4
5
6
7
8
9

In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False,
                                              weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  #tf.keras.layers.MaxPool2D(pool_size =(2, 2), strides = None, padding='valid', data_format=None, **kwargs)
  tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                330       
Total params: 2,626,986
Trainable params: 369,002
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 4


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
epochs = 10

history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=epochs,
                    validation_data=val_generator,
                    validation_steps=len(val_generator))

Epoch 1/10
26/26 [==============================] - 538s 21s/step - loss: 2.2666 - accuracy: 0.2819 - val_loss: 1.4475 - val_accuracy: 0.4817
Epoch 2/10
26/26 [==============================] - 6s 239ms/step - loss: 1.0691 - accuracy: 0.6649 - val_loss: 1.2280 - val_accuracy: 0.5966
Epoch 3/10
26/26 [==============================] - 6s 236ms/step - loss: 0.7495 - accuracy: 0.7659 - val_loss: 1.0167 - val_accuracy: 0.6504
Epoch 4/10
26/26 [==============================] - 6s 242ms/step - loss: 0.5558 - accuracy: 0.8191 - val_loss: 0.9340 - val_accuracy: 0.6773
Epoch 5/10
26/26 [==============================] - 6s 244ms/step - loss: 0.4194 - accuracy: 0.8463 - val_loss: 0.7442 - val_accuracy: 0.7531
Epoch 6/10
26/26 [==============================] - 6s 236ms/step - loss: 0.3083 - accuracy: 0.9038 - val_loss: 0.7391 - val_accuracy: 0.7995
Epoch 7/10
26/26 [==============================] - 6s 243ms/step - loss: 0.1904 - accuracy: 0.9570 - val_loss: 0.7946 - val_accuracy: 0.7922
Epoch 

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  155


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                330       
Total params: 2,626,986
Trainable params: 2,231,594
Non-trainable params: 395,392
_________________________________________________________________


In [ ]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 60


In [ ]:
history_fine = model.fit(train_generator,
                         steps_per_epoch=len(train_generator),
                         epochs=20,
                         validation_data=val_generator,
                         validation_steps=len(val_generator))

Epoch 1/20
26/26 [==============================] - 7s 280ms/step - loss: 1.9450 - accuracy: 0.5475 - val_loss: 0.8182 - val_accuracy: 0.7995
Epoch 2/20
26/26 [==============================] - 7s 252ms/step - loss: 0.8534 - accuracy: 0.7296 - val_loss: 0.8526 - val_accuracy: 0.7897
Epoch 3/20
26/26 [==============================] - 6s 243ms/step - loss: 0.4430 - accuracy: 0.8500 - val_loss: 0.8756 - val_accuracy: 0.7922
Epoch 4/20
26/26 [==============================] - 7s 254ms/step - loss: 0.2868 - accuracy: 0.9153 - val_loss: 0.8678 - val_accuracy: 0.7873
Epoch 5/20
26/26 [==============================] - 7s 252ms/step - loss: 0.2194 - accuracy: 0.9425 - val_loss: 0.8525 - val_accuracy: 0.8020
Epoch 6/20
26/26 [==============================] - 7s 253ms/step - loss: 0.1690 - accuracy: 0.9583 - val_loss: 0.8278 - val_accuracy: 0.8117
Epoch 7/20
26/26 [==============================] - 6s 248ms/step - loss: 0.1291 - accuracy: 0.9728 - val_loss: 0.8199 - val_accuracy: 0.8068
Epoch 

In [ ]:
model.save("detecthandee.h5")